In [ ]:
# cnn model and accuracy - porosity tensor dataset
# Import packages / libraries

import os
from os.path import dirname, join as pjoin
import scipy.io
import tensorflow as tf
import torch

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

from os import makedirs
from os import listdir
from shutil import copyfile
from random import seed
from random import random

# baseline model for the classification dataset
import sys
from matplotlib import pyplot
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from sklearn.model_selection import KFold, StratifiedKFold
from tensorflow.keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator

#!pip install opencv-python
#!pip install lime 

import numpy as np
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from skimage.color import gray2rgb
from skimage.color import rgb2gray
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import inception_v3 as inc_net
from lime import lime_image
import pandas as pd
import glob
from tensorflow.keras.models import load_model
from sklearn.metrics import r2_score
from cv2 import cv2
from keras.preprocessing.image import ImageDataGenerator
from skimage.segmentation import mark_boundaries

In [ ]:
## LIME ANALYSIS
## Import required libraries / packages

import numpy as np
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
import math
import scipy
import scipy.io
from PIL import Image
from scipy import ndimage
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
# import tensorflow_addons as tfa
import pydot
import pydotplus
import graphviz
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import Model
import random
from keras.models import load_model
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import glob
import os
import pandas as pd

import types
from lime.utils.generic_utils import has_arg
from skimage.segmentation import felzenszwalb, slic, quickshift
import copy
from functools import partial

import sklearn
import sklearn.preprocessing
from sklearn.utils import check_random_state
from skimage.color import gray2rgb
from tqdm.auto import tqdm

import scipy.ndimage as ndi
from skimage.segmentation._quickshift_cy import _quickshift_cython

from lime import lime_base
from lime.wrappers.scikit_image import SegmentationAlgorithm

import skimage
from matplotlib import colors
from skimage.segmentation import mark_boundaries, find_boundaries
from skimage.morphology import dilation,square
from collections import Counter

In [ ]:
# organize dataset into a numpy structure
# Extract input spectrogram data from .mat files 

np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
# sets print options to 5 decimal places

TENSOR_INPUT_DATA_COMBINED = torch.ones((72,6,129,15))
TENSOR_INPUT_DATA_PRINTING = torch.ones((72,4,129,15))
TENSOR_INPUT_DATA_MILLING = torch.ones((72,2,129,15))
    
    # 72 - Total number of data points for binary classification
    # 6 - Channels (pertaining to each spectrogram)
    # 129  - Freuqency bands for each spectrogram
    # 15 - Time steps

voxel_number = 0

# assign directory
# directory = 'files'
# iterate over files in that directory
# for filename in os.listdir(directory):


directory = (r"C:\Users\adithyaa\Downloads\PhD Downloads\Porosity_Project_ISEN658\Machine_Learning_model_results\Spectrograms_Tensor_data\PYTHON")
# Change this directory location to your folder that contains the 72 .mat files from Tensor_data_72_voxels.zip corresponding to Sample 1.
# When working with Sample 4 (Tensor_data_90_voxels_sample4.zip), extract the 90 files in a folder, and update the directory. In addition, 
# update the dimensions in the below for loop (and in other places as required) as each tensor in Sample 4 is of dimension 129x6x4, 
# ie, 129 frequency bands, 6 time steps, 4 spectrograms pertaining to only printing cycle. 

for filename in os.listdir(directory):
    # print(filename)
    mat_fname = pjoin(directory, filename)
    Tensor_data_voxel = scipy.io.loadmat(mat_fname)

    data = list(Tensor_data_voxel. items())
    tensor_array = np.array(data, dtype=object)

    # The spectrogram data is captured in a list from one of the elements of tensor_array
    # Shape of tensor_array is (4,2) and all spectrogram data (129x15x6) is stored in tensor_array[3][1]

    spec_list = list(tensor_array[3][1])

    for i in range(len(spec_list)):
        for j in range(15):
            for k in range(6):
                TENSOR_INPUT_DATA_COMBINED[voxel_number,k,i,j] = spec_list[i][j][k]
                    
                    
    TENSOR_INPUT_DATA_PRINTING = TENSOR_INPUT_DATA_COMBINED[:,(1,2,4,5),:,:]
    TENSOR_INPUT_DATA_MILLING = TENSOR_INPUT_DATA_COMBINED[:,(0,3),:,:]                

    voxel_number = voxel_number + 1
    

y_target = np.zeros(72)
y_target[4:10] = 1
y_target[13:18] = 1
y_target[22:34] = 1
y_target[37:41] = 1
y_target[43:46] = 1
y_target[47] = 1
y_target[57] = 1
y_target[59] = 1
y_target[70] = 1

print("---------------------")
print(TENSOR_INPUT_DATA_COMBINED.shape)
print(TENSOR_INPUT_DATA_COMBINED)
print(TENSOR_INPUT_DATA_PRINTING.shape)
print(TENSOR_INPUT_DATA_PRINTING)
print(TENSOR_INPUT_DATA_MILLING.shape)
print(TENSOR_INPUT_DATA_MILLING)
print("---------------------")
print(y_target)
print("---------------------")
data_order = np.arange(72)
np.random.shuffle(data_order)
print(data_order)

---------------------
torch.Size([72, 6, 129, 15])
tensor([[[[3.9972e-01, 2.7345e-01, 1.9331e-01,  ..., 4.0026e-02,
           1.4011e-02, 5.2142e-03],
          [1.5534e-01, 9.1346e-02, 2.7695e-02,  ..., 6.1477e-02,
           3.4975e-02, 5.1146e-02],
          [4.8935e-02, 4.4440e-02, 8.1369e-03,  ..., 2.1981e-02,
           3.2418e-02, 4.0163e-02],
          ...,
          [3.3482e-03, 7.8327e-03, 1.2898e-02,  ..., 1.9981e-02,
           7.0467e-03, 1.7296e-02],
          [7.0541e-03, 7.8069e-03, 1.8567e-02,  ..., 3.4912e-03,
           1.6951e-02, 9.5547e-03],
          [6.0237e-03, 1.9540e-02, 3.1549e-02,  ..., 3.4602e-02,
           4.3607e-02, 5.1279e-03]],

         [[1.1205e-02, 6.9918e-02, 1.6245e-01,  ..., 3.3511e-02,
           2.9558e-02, 1.6605e-01],
          [6.5787e-02, 2.9396e-02, 3.7733e-02,  ..., 5.1377e-02,
           2.9544e-02, 9.7831e-02],
          [5.2401e-02, 2.8427e-02, 3.7980e-02,  ..., 4.8940e-02,
           4.5427e-02, 4.1293e-02],
          ...,
        

In [ ]:
# Saving data as .npy files
NUMPY_INPUT_DATA = TENSOR_INPUT_DATA_COMBINED.numpy()
NUMPY_INPUT_DATA_printing = TENSOR_INPUT_DATA_PRINTING.numpy()
NUMPY_INPUT_DATA_milling = TENSOR_INPUT_DATA_MILLING.numpy()

np.save(r"C:\Users\adithyaa\Downloads\PhD Downloads\Porosity_Project_ISEN658\Machine_Learning_model_results\Spectrograms_Tensor_data\INPUT_MATRIX_Combined_data", NUMPY_INPUT_DATA)
np.save(r"C:\Users\adithyaa\Downloads\PhD Downloads\Porosity_Project_ISEN658\Machine_Learning_model_results\Spectrograms_Tensor_data\INPUT_MATRIX_Printing_data", NUMPY_INPUT_DATA_printing)
np.save(r"C:\Users\adithyaa\Downloads\PhD Downloads\Porosity_Project_ISEN658\Machine_Learning_model_results\Spectrograms_Tensor_data\INPUT_MATRIX_Milling_data", NUMPY_INPUT_DATA_milling)
np.save(r"C:\Users\adithyaa\Downloads\PhD Downloads\Porosity_Project_ISEN658\Machine_Learning_model_results\Spectrograms_Tensor_data\TARGET_MATRIX", y_target)

In [ ]:
# CNN Model Architecture - Printing and Milling combined cycles

NUMPY_INPUT_DATA = TENSOR_INPUT_DATA_COMBINED.numpy()

X_train = NUMPY_INPUT_DATA[data_order[0:64],:,:,:]
y_train = y_target[data_order[0:64]]
X_test = NUMPY_INPUT_DATA[data_order[64:72],:,:,:]
y_test = y_target[data_order[64:72]]

# Define the K-fold Cross Validator
kfold = KFold(n_splits=8, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
train_acc_per_fold = []
train_loss_per_fold = []
val_acc_per_fold = []
val_loss_per_fold = []
test_acc_per_fold = []
test_loss_per_fold = []

for train, val in kfold.split(X_train, y_train):
    
    train_acc = 0
    val_acc = 0
    test_acc = 0
    # Define the model architecture
    while(train_acc < 0.5 or val_acc < 0.5 or test_acc < 0.5):
        model = Sequential()
        model.add(Conv2D(6, (7, 7), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(6, 129, 15)))
        model.add(MaxPooling2D((2, 2)))
        model.add(Conv2D(1, (7, 7), activation='relu', kernel_initializer='he_uniform', padding='same'))
        model.add(MaxPooling2D((2, 2)))
        # model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
        # model.add(MaxPooling2D((2, 2)))
        model.add(Flatten())
        model.add(Dense(6, activation='relu', kernel_initializer='he_uniform'))
        model.add(Dense(1, activation='sigmoid'))
    
        # Compile the model
        opt = SGD(lr=0.001, momentum=0.9)
        model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    
        # Generate a print
        print('------------------------------------------------------------------------')
        print(f'Training for fold {fold_no} ...')

        # Fit data to model
        history = model.fit(X_train[train], y_train[train], epochs=500, verbose=0)
        #train_acc_per_fold.append(history.history['accuracy'])
        #train_loss_per_fold.append(history.history['loss'])
        #summarize_diagnostics(fold_no, history)
        model.save('porosity_bin_model_fold_no_'+ str(fold_no) +'.h5')
    
        # Generate generalization metrics
        train_scores = model.evaluate(X_train[train], y_train[train], verbose=0)
        print(f'Train Score for fold {fold_no}: {model.metrics_names[0]} of {train_scores[0]}; {model.metrics_names[1]} of {train_scores[1]*100}%')
        train_acc = train_scores[1]
        
        val_scores = model.evaluate(X_train[val], y_train[val], verbose=0)
        print(f'Val Score for fold {fold_no}: {model.metrics_names[0]} of {val_scores[0]}; {model.metrics_names[1]} of {val_scores[1]*100}%')
        val_acc = val_scores[1]
        
        test_scores = model.evaluate(X_test, y_test, steps=len(X_test), verbose = 0)
        print(f'Test data Score for fold {fold_no}: {model.metrics_names[0]} of {test_scores[0]}; {model.metrics_names[1]} of {test_scores[1]*100}%')
        test_acc = test_scores[1]
    
    # Increase fold number
    fold_no = fold_no + 1
    train_acc_per_fold.append(train_scores[1] * 100)
    train_loss_per_fold.append(train_scores[0])
    val_acc_per_fold.append(val_scores[1] * 100)
    val_loss_per_fold.append(val_scores[0])
    test_acc_per_fold.append(test_scores[1] * 100)
    test_loss_per_fold.append(test_scores[0])

------------------------------------------------------------------------
Training for fold 1 ...
Train Score for fold 1: loss of 0.0018056032713502645; accuracy of 100.0%
Val Score for fold 1: loss of 1.5462905168533325; accuracy of 50.0%
Test data Score for fold 1: loss of 0.20194101333618164; accuracy of 87.5%
------------------------------------------------------------------------
Training for fold 2 ...
Train Score for fold 2: loss of 0.0017346928361803293; accuracy of 100.0%
Val Score for fold 2: loss of 2.1656148433685303; accuracy of 50.0%
Test data Score for fold 2: loss of 0.42851752042770386; accuracy of 87.5%
------------------------------------------------------------------------
Training for fold 3 ...
Train Score for fold 3: loss of 0.6906078457832336; accuracy of 53.57142686843872%
Val Score for fold 3: loss of 0.6787701845169067; accuracy of 62.5%
Test data Score for fold 3: loss of 0.7119154930114746; accuracy of 37.5%
--------------------------------------------------

In [ ]:
# CNN Model Architecture - Printing cycles

NUMPY_INPUT_DATA_printing = TENSOR_INPUT_DATA_PRINTING.numpy()

X_train_printing = NUMPY_INPUT_DATA_printing[data_order[0:64],:,:,:]
y_train_printing = y_target[data_order[0:64]]
X_test_printing = NUMPY_INPUT_DATA_printing[data_order[64:72],:,:,:]
y_test_printing = y_target[data_order[64:72]]

# Define the K-fold Cross Validator
kfold = KFold(n_splits=8, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
train_acc_per_fold_printing = []
train_loss_per_fold_printing = []
val_acc_per_fold_printing = []
val_loss_per_fold_printing = []
test_acc_per_fold_printing = []
test_loss_per_fold_printing = []

for train, val in kfold.split(X_train_printing, y_train_printing):
    
    train_acc = 0
    val_acc = 0
    test_acc = 0
    # Define the model architecture
    while(train_acc < 0.5 or val_acc < 0.5 or test_acc < 0.5):
        model = Sequential()
        model.add(Conv2D(6, (7, 7), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(4, 129, 15)))
        model.add(MaxPooling2D((2, 2)))
        model.add(Conv2D(1, (7, 7), activation='relu', kernel_initializer='he_uniform', padding='same'))
        model.add(MaxPooling2D((2, 2)))
        # model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
        # model.add(MaxPooling2D((2, 2)))
        model.add(Flatten())
        model.add(Dense(6, activation='relu', kernel_initializer='he_uniform'))
        model.add(Dense(1, activation='sigmoid'))
    
        # Compile the model
        opt = SGD(lr=0.001, momentum=0.9)
        model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    
        # Generate a print
        print('------------------------------------------------------------------------')
        print(f'Training for fold {fold_no} ...')

        # Fit data to model
        history = model.fit(X_train_printing[train], y_train_printing[train], epochs=500, verbose=0)
        #train_acc_per_fold.append(history.history['accuracy'])
        #train_loss_per_fold.append(history.history['loss'])
        #summarize_diagnostics(fold_no, history)
        model.save('porosity_bin_model_printing_fold_no_'+ str(fold_no) +'.h5')
    
        # Generate generalization metrics
        train_scores = model.evaluate(X_train_printing[train], y_train_printing[train], verbose=0)
        print(f'Train Score for fold {fold_no}: {model.metrics_names[0]} of {train_scores[0]}; {model.metrics_names[1]} of {train_scores[1]*100}%')
        train_acc = train_scores[1]
        
        val_scores = model.evaluate(X_train_printing[val], y_train_printing[val], verbose=0)
        print(f'Val Score for fold {fold_no}: {model.metrics_names[0]} of {val_scores[0]}; {model.metrics_names[1]} of {val_scores[1]*100}%')
        val_acc = val_scores[1]
        
        test_scores = model.evaluate(X_test_printing, y_test_printing, steps=len(X_test_printing), verbose = 0)
        print(f'Test data Score for fold {fold_no}: {model.metrics_names[0]} of {test_scores[0]}; {model.metrics_names[1]} of {test_scores[1]*100}%')
        test_acc = test_scores[1]
    
    # Increase fold number
    fold_no = fold_no + 1
    train_acc_per_fold_printing.append(train_scores[1] * 100)
    train_loss_per_fold_printing.append(train_scores[0])
    val_acc_per_fold_printing.append(val_scores[1] * 100)
    val_loss_per_fold_printing.append(val_scores[0])
    test_acc_per_fold_printing.append(test_scores[1] * 100)
    test_loss_per_fold_printing.append(test_scores[0])

------------------------------------------------------------------------
Training for fold 1 ...
Train Score for fold 1: loss of 0.004864283837378025; accuracy of 100.0%
Val Score for fold 1: loss of 2.8012280464172363; accuracy of 62.5%
Test data Score for fold 1: loss of 0.3190351724624634; accuracy of 75.0%
------------------------------------------------------------------------
Training for fold 2 ...
Train Score for fold 2: loss of 0.003608131315559149; accuracy of 100.0%
Val Score for fold 2: loss of 1.0540189743041992; accuracy of 62.5%
Test data Score for fold 2: loss of 0.3158787488937378; accuracy of 75.0%
------------------------------------------------------------------------
Training for fold 3 ...
Train Score for fold 3: loss of 0.0031603898387402296; accuracy of 100.0%
Val Score for fold 3: loss of 1.1288626194000244; accuracy of 87.5%
Test data Score for fold 3: loss of 0.5363999605178833; accuracy of 62.5%
---------------------------------------------------------------

Train Score for fold 5: loss of 0.005274743773043156; accuracy of 100.0%
Val Score for fold 5: loss of 3.0437378883361816; accuracy of 0.0%
Test data Score for fold 5: loss of 0.36486679315567017; accuracy of 87.5%
------------------------------------------------------------------------
Training for fold 5 ...
Train Score for fold 5: loss of 0.00454071955755353; accuracy of 100.0%
Val Score for fold 5: loss of 5.658059120178223; accuracy of 12.5%
Test data Score for fold 5: loss of 0.509437620639801; accuracy of 87.5%
------------------------------------------------------------------------
Training for fold 5 ...
Train Score for fold 5: loss of 0.0032686723861843348; accuracy of 100.0%
Val Score for fold 5: loss of 4.751274108886719; accuracy of 12.5%
Test data Score for fold 5: loss of 1.373470664024353; accuracy of 62.5%
------------------------------------------------------------------------
Training for fold 5 ...
Train Score for fold 5: loss of 0.0026414154563099146; accuracy of 1

In [ ]:
# CNN Model Architecture - Milling cycle

NUMPY_INPUT_DATA_milling = TENSOR_INPUT_DATA_MILLING.numpy()

X_train_milling = NUMPY_INPUT_DATA_milling[data_order[0:64],:,:,:]
y_train_milling = y_target[data_order[0:64]]
X_test_milling = NUMPY_INPUT_DATA_milling[data_order[64:72],:,:,:]
y_test_milling = y_target[data_order[64:72]]

# Define the K-fold Cross Validator
kfold = KFold(n_splits=8, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
train_acc_per_fold_milling = []
train_loss_per_fold_milling = []
val_acc_per_fold_milling = []
val_loss_per_fold_milling = []
test_acc_per_fold_milling = []
test_loss_per_fold_milling = []

for train, val in kfold.split(X_train_milling, y_train_milling):
    
    train_acc = 0
    val_acc = 0
    test_acc = 0
    # Define the model architecture
    while(train_acc < 0.5 or val_acc < 0.5 or test_acc < 0.5):
        model = Sequential()
        model.add(Conv2D(6, (7, 7), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(2, 129, 15)))
        model.add(MaxPooling2D((2, 2)))
        model.add(Conv2D(1, (7, 7), activation='relu', kernel_initializer='he_uniform', padding='same'))
        # model.add(MaxPooling2D((2, 2)))
        # model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
        # model.add(MaxPooling2D((2, 2)))
        model.add(Flatten())
        model.add(Dense(6, activation='relu', kernel_initializer='he_uniform'))
        model.add(Dense(1, activation='sigmoid'))
    
        # Compile the model
        opt = SGD(lr=0.001, momentum=0.9)
        model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    
        # Generate a print
        print('------------------------------------------------------------------------')
        print(f'Training for fold {fold_no} ...')

        # Fit data to model
        history = model.fit(X_train_milling[train], y_train_milling[train], epochs=500, verbose=0)
        #train_acc_per_fold.append(history.history['accuracy'])
        #train_loss_per_fold.append(history.history['loss'])
        #summarize_diagnostics(fold_no, history)
        model.save('porosity_bin_model_milling_fold_no_'+ str(fold_no) +'.h5')
    
        # Generate generalization metrics
        train_scores = model.evaluate(X_train_milling[train], y_train_milling[train], verbose=0)
        print(f'Train Score for fold {fold_no}: {model.metrics_names[0]} of {train_scores[0]}; {model.metrics_names[1]} of {train_scores[1]*100}%')
        train_acc = train_scores[1]
        
        val_scores = model.evaluate(X_train_milling[val], y_train_milling[val], verbose=0)
        print(f'Val Score for fold {fold_no}: {model.metrics_names[0]} of {val_scores[0]}; {model.metrics_names[1]} of {val_scores[1]*100}%')
        val_acc = val_scores[1]
        
        test_scores = model.evaluate(X_test_milling, y_test_milling, steps=len(X_test_milling), verbose = 0)
        print(f'Test data Score for fold {fold_no}: {model.metrics_names[0]} of {test_scores[0]}; {model.metrics_names[1]} of {test_scores[1]*100}%')
        test_acc = test_scores[1]
    
    # Increase fold number
    fold_no = fold_no + 1
    train_acc_per_fold_milling.append(train_scores[1] * 100)
    train_loss_per_fold_milling.append(train_scores[0])
    val_acc_per_fold_milling.append(val_scores[1] * 100)
    val_loss_per_fold_milling.append(val_scores[0])
    test_acc_per_fold_milling.append(test_scores[1] * 100)
    test_loss_per_fold_milling.append(test_scores[0])

------------------------------------------------------------------------
Training for fold 1 ...
Train Score for fold 1: loss of 0.33108144998550415; accuracy of 96.42857313156128%
Val Score for fold 1: loss of 0.7014734745025635; accuracy of 50.0%
Test data Score for fold 1: loss of 1.1977633237838745; accuracy of 25.0%
------------------------------------------------------------------------
Training for fold 1 ...
Train Score for fold 1: loss of 0.021736200898885727; accuracy of 100.0%
Val Score for fold 1: loss of 1.248964548110962; accuracy of 50.0%
Test data Score for fold 1: loss of 1.471377968788147; accuracy of 62.5%
------------------------------------------------------------------------
Training for fold 2 ...
Train Score for fold 2: loss of 0.052628934383392334; accuracy of 100.0%
Val Score for fold 2: loss of 1.0996490716934204; accuracy of 50.0%
Test data Score for fold 2: loss of 1.2982370853424072; accuracy of 37.5%
-------------------------------------------------------

In [ ]:
# Actual predictions count Verification block

X_test = NUMPY_INPUT_DATA[data_order[64:72],:,:,:]
y_test = y_target[data_order[64:72]]

LX = NUMPY_INPUT_DATA[data_order,:,:,:]
LY = y_target[data_order]

for i in range(1,9):
    model_name = 'porosity_bin_model_fold_no_' + str(i) + '.h5'
    model = load_model(model_name)
    pred = model.predict(LX)
    # LY = np.array([1 if a == 0 else 0 for a in Y])
    pred_class = [0 if a < 0.5 else 1 for a in pred]
    correct_X_ind = list(np.where(pred_class==LY)[0])
    print('------------------------------------------------------------------------')
    print(f'Correct predictions in fold {i} ...')
    print(len(correct_X_ind))  
    pred_test = model.predict(X_test)
    pred_class_test = [0 if a < 0.5 else 1 for a in pred_test]
    correct_X_ind = list(np.where(pred_class_test==y_test)[0])
    print(f'Correct Test data predictions for fold {i} ...')
    print(len(correct_X_ind))

------------------------------------------------------------------------
Correct predictions in fold 1 ...
61
Correct Test data predictions for fold 1 ...
3
------------------------------------------------------------------------
Correct predictions in fold 2 ...
63
Correct Test data predictions for fold 2 ...
3
------------------------------------------------------------------------
Correct predictions in fold 3 ...
38
Correct Test data predictions for fold 3 ...
2
------------------------------------------------------------------------
Correct predictions in fold 4 ...
47
Correct Test data predictions for fold 4 ...
4
------------------------------------------------------------------------
Correct predictions in fold 5 ...
38
Correct Test data predictions for fold 5 ...
2
------------------------------------------------------------------------
Correct predictions in fold 6 ...
63
Correct Test data predictions for fold 6 ...
4
----------------------------------------------------------